# Import software libraries

# Load and preview the data

# Check the shape of the data

# Check the data types

# Explore the distribution of the target variable

- You need to plot the distribution of the target column (term_deposit) and explain what is the problem of the distribution.

# Apply oversampling or SMOTE to the target label

# Normalize the data

# Standardlize the data

# Plot the PCA